In [1]:
import nest_asyncio
from crewai import LLM
from dotenv import load_dotenv
import os
import asyncio
from echo.constants import (
    DISCOVERY,
    DEMO,
    PRICING,
    NEGOTIATION
)


nest_asyncio.apply()

load_dotenv()

llm = LLM(
    model=os.getenv("FIREWORKS_MODEL_NAME"),
    base_url="https://api.fireworks.ai/inference/v1",
    api_key=os.getenv("FIREWORKS_API_KEY"),
)

In [18]:
from echo.runner import create_or_get_seller

inputs = {
    "seller": "foundit.in",
    "num_buyers": 10,
}

seller = asyncio.run(create_or_get_seller(inputs))

Creating seller for foundit.in
Adding Seller: foundit.in Data


In [19]:
from echo.runner import make_call


buyer_inputs = {
    **inputs,
    **seller,
    'n_competitors': 10,
}

clients = seller['seller_clients']['clients']

In [20]:
seller['seller_clients']['clients']

['manipaltechnologies.com',
 'Monster India',
 'Quess Corp',
 'Third Bridge Group Limited']

In [21]:
discovery_calls_data = asyncio.run(make_call(DISCOVERY, seller['seller_clients']['clients'], buyer_inputs))

Making discovery call for ['manipaltechnologies.com', 'Monster India', 'Quess Corp', 'Third Bridge Group Limited']
Getting analysis Data
Number of Clients:  4


Getting Data:   0%|          | 0/4 [00:00<?, ?it/s]2025-02-18 15:47:23,069 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Getting Data for manipaltechnologies.com
Data not found for client: manipaltechnologies.com
Data not found for client: manipaltechnologies.com
Adding Buyer: manipaltechnologies.com Data


2025-02-18 15:47:38,683 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding discovery Transcript Data to Vector Store


...

2025-02-18 15:49:14,929 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding discovery Transcript Data to Vector Store


2025-02-18 15:49:20,502 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding Analysis Data to Vector Store


Getting Data: 100%|██████████| 4/4 [02:03<00:00, 30.86s/it]


In [6]:
demo_calls_data = asyncio.run(make_call(DEMO, clients, buyer_inputs))

Making demo call for ['Cardinal Health', 'BMC Software', 'Grant Thornton', 'Coca Cola', 'Equinix', 'Caterpillar, Inc.']
Getting analysis Data
Number of Clients:  6


Getting Data:   0%|          | 0/6 [00:00<?, ?it/s]2025-02-17 13:08:16,467 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Getting Data for Cardinal Health
Adding Research Data to Vector Store


2025-02-17 13:08:19,429 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding demo Transcript Data to Vector Store


...

2025-02-17 13:10:41,188 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding demo Transcript Data to Vector Store


2025-02-17 13:10:47,537 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Retrying due to error: Failed to convert text into a Pydantic model due to the following error: Instructor does not support multiple tool calls, use List[Model] instead
Retrying due to error: Failed to convert text into a Pydantic model due to the following error: Instructor does not support multiple tool calls, use List[Model] instead
Adding Analysis Data to Vector Store


Getting Data: 100%|██████████| 6/6 [03:22<00:00, 33.72s/it]


In [7]:
pricing_call_data = asyncio.run(make_call(PRICING, clients, buyer_inputs))

Making pricing call for ['Cardinal Health', 'BMC Software', 'Grant Thornton', 'Coca Cola', 'Equinix', 'Caterpillar, Inc.']
Getting analysis Data
Number of Clients:  6


Getting Data:   0%|          | 0/6 [00:00<?, ?it/s]2025-02-17 13:11:38,780 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Getting Data for Cardinal Health
Data exists but pricing transcript not found
Adding pricing Transcript Data to Vector Store


2025-02-17 13:11:45,513 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding Analysis Data to Vector Store


...

Getting Data:  83%|████████▎ | 5/6 [00:49<00:09,  9.53s/it]2025-02-17 13:12:27,964 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Getting Data for Caterpillar, Inc.
Data exists but pricing transcript not found
Adding pricing Transcript Data to Vector Store


2025-02-17 13:12:32,611 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding Analysis Data to Vector Store


Getting Data: 100%|██████████| 6/6 [00:57<00:00,  9.52s/it]


In [8]:
negotiation_call_data = asyncio.run(make_call(NEGOTIATION, clients, buyer_inputs))

Making negotiation call for ['Cardinal Health', 'BMC Software', 'Grant Thornton', 'Coca Cola', 'Equinix', 'Caterpillar, Inc.']
Getting analysis Data
Number of Clients:  6


Getting Data:   0%|          | 0/6 [00:00<?, ?it/s]2025-02-17 13:12:35,921 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Getting Data for Cardinal Health
Data exists but pricing transcript not found
Adding negotiation Transcript Data to Vector Store


2025-02-17 13:12:40,906 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding Analysis Data to Vector Store


...

Getting Data:  83%|████████▎ | 5/6 [00:44<00:08,  8.99s/it]2025-02-17 13:13:20,315 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Getting Data for Caterpillar, Inc.
Data exists but pricing transcript not found
Adding negotiation Transcript Data to Vector Store


2025-02-17 13:13:26,756 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding Analysis Data to Vector Store


Getting Data: 100%|██████████| 6/6 [00:54<00:00,  9.15s/it]


## Queries

In [9]:
# len(index._storage_context.vector_store._collection.get()['metadatas'])

In [10]:
from echo.queries import get_queries

seller = "Whatfix"
queries = get_queries(seller)

In [11]:
print("Total queries: ", len(queries))

Total queries:  4


In [12]:
queries.keys()

dict_keys(['discovery', 'demo', 'pricing', 'negotiation'])

In [13]:
# vector_index.storage_context.vector_store._collection.get()['metadatas']

In [14]:
from echo.query_executor import arun_queries
from echo.query_executor import ResponseFormat
from echo.query_executor import ContextExtractionMode
import nest_asyncio
import asyncio

nest_asyncio.apply()


inputs = {"buyer": "BMC Software", "company_size": "Enterprise"}

responses = asyncio.run(
    arun_queries(
        queries,
        inputs=inputs,
        response_format=ResponseFormat.MARKDOWN,
        context_extraction_mode=ContextExtractionMode.QUERY_ENGINE,
    )
)

Running queries for call type discovery


Running Queries:   0%|          | 0/8 [00:00<?, ?it/s]

Running sub query What all pieces of information about the buyer are uncovered in discovery calls of successful deals?
Sub query context {'query': 'What all pieces of information about the buyer are uncovered in discovery calls of successful deals?', 'context': 'Relevant Context:\nThe pain points, objections, time lines, success indicators, budget constraints, competition, and decision committee members of the buyer are uncovered in discovery calls of successful deals.'}
Running sub query What all piece of information have we learnt about the buyer from discovery till now?


2025-02-17 13:13:35,707 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Sub query context {'query': 'What all piece of information have we learnt about the buyer from discovery till now?', 'context': 'Relevant Context:\nThe buyer, BMC Software, is dealing with complex IT problems that hinder their ability to minimize cost and effort. They are struggling with remote monitoring and management of host systems. BMC Software is unsure if digital adoption platforms can meet their specific requirements and they need a customized solution that can integrate with existing systems and provide advanced analytics and performance support. They are open to exploring different pricing models and have not set a specific budget yet. The decision committee includes Ayman Sayed (CEO), John Moores (Founder), and IT department stakeholders. The buyer is looking for a digital adoption platform that can provide a significant reduction in cost and effort, as well as an improvement in user adoption and satisfaction. They have a timeline of 6-12 months for implementing a digital ad

...

Trying to parse with Ast...
Running sub query What are the closing tactics that have been successful in the past?


2025-02-17 13:15:42,007 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Sub query context {'query': 'What are the closing tactics that have been successful in the past?', 'context': 'Relevant Context:\nThe closing tactics that have been successful in the past include finalizing the pricing and implementation plan, sending over a proposal within the next 24 hours, offering a detailed quote, scheduling a follow-up call, assigning a customer success manager, providing a clear next step, and sending a formal proposal.'}
Running final query What final closing tactics can be used for their account?


Running Queries: 100%|██████████| 4/4 [00:16<00:00,  4.06s/it]

Trying to parse with Ast...


In [15]:
len(responses.keys())

19

In [16]:
responses["Info To Cover"]

{'response': "**Missing Information to Uncover in Upcoming Calls**\n==============================================\n\n### Section 1: Buyer's Current Challenges and Pain Points\n\n* More details on the specific challenges faced by BMC Software's IT operations team and how they affect digital transformation\n* Quantification of the costs associated with user support and training, and the impact on their business\n* Additional pain points related to remote monitoring and management of host systems\n\n### Section 2: Budget and Pricing\n\n* A clear understanding of the budget allocated for the digital adoption platform\n* Details on the preferred pricing model and any flexibility in terms of pricing structure\n* Comparison of Whatfix's pricing model with competitors like WalkMe, Pendo, and Userpilot\n\n### Section 3: Decision Committee and Timeline\n\n* More information on the decision-making process and the roles of each stakeholder (Ayman Sayed, John Moores, and IT department stakeholders

## Dynamic Research

In [1]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [3]:
from echo.runner import create_or_get_seller

inputs = {
    "seller": "foundit.in",
    "num_buyers": 10,
}

seller = asyncio.run(create_or_get_seller(inputs))

2025-02-18 15:26:58,585 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Creating seller for foundit.in
Adding Seller: foundit.in Data


In [22]:
import json


buyer_research = json.load(open(".echo_storage/runs/foundit.in/manipaltechnologies.com.json"))

In [24]:
buyer_research['seller_research']['description']

"India's leading job portal that helps businesses find and hire freelancers, temporary staff, and other talent faster."

In [27]:
from echo.utils import get_crew, add_pydantic_structure
from pydantic import BaseModel, Field
from typing import List


class Query(BaseModel):
    name: str = Field(..., title="What is the topic of the research information to be extracted about a client?")
    description: str = Field(..., title="Purpose of the research query. Why do we need this research information about a client?")
    questions: List[str] = Field(..., title="What all questions need to be answered to get the required information about this research topic about the client?")


class Queries(BaseModel):
    queries: List[Query] = Field(..., title="List of research queries to be executed to gather information about a client")


agent_templates = {
    "Account Executive": dict(
        role="Account Executive",
        goal="In-depth research about a company to understand their needs and challenges",
        backstory=(
            "You are an Account Executive at {seller}, {seller_description}"
            "You are responsible to do account research and prepare a discovery checklist before the first discovery call.\n"
            "Your goal is to discover research and align value prop specific to the product you're selling so that you can align value proposition better to the account you're targeting"
        ),
    ),
}

task_templates = {
    "Research Query Planning": dict(
        name="Research Query Planning", 
        description=(
            "You need to plan the research queries to be executed to gather information about {buyer} based on the {seller} research information."
            "Below is the information about {seller}:\n\n{seller_info}"
            "Based on the information about {seller}, you need to plan the research queries that need to be extracted for the Account Executive of {seller} to do the following the order of priority - \n"
            "1. Align the value proposition of the products with the needs and challenges of the client\n"
            "2. Help {seller} maximize their business goals as a company\n"
            "3. Understand the needs and challenges of the {buyer}\n"
            
            "Below is the information about {buyer}:\n\n{buyer_info}\n"
            "Extract the research queries that need to be executed to gather information about {buyer} based on the {seller} research information that will help the Account Executive of {seller} to understand the needs and challenges of the client, align the value proposition of the products with the needs and challenges of the client, and help {seller} maximize their business goals as a company."
            
        ), 
        expected_output=(
            "The research queries to be executed to gather information about a client based on the {seller} research information."
            "The response should conform to the provided schema.\n"
            "You need to extract the following information in the following pydantic structure -\n"
            "{pydantic_structure}\n"
            "Make sure there are no comments in the response JSON and it should be a valid JSON."
        ),
        output_pydantic=Queries,
        agent='Account Executive', 
    )
}

inputs = {
    "seller": "Whatfix",
    "seller_description": seller['seller_research']['description'],
    "seller_info": seller['seller_research'],
    "buyer": "Manipal Technologies",
    "buyer_info": json.dumps(buyer_research['buyer_research'], indent=2) + "\nPains and Objections:\n" + json.dumps(buyer_research['anticipated_qopcs'], indent=2),
}

crew = get_crew(
    agent_templates=agent_templates, 
    task_templates=task_templates
)
add_pydantic_structure(crew, inputs)

2025-02-18 15:59:45,130 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [28]:
response = asyncio.run(crew.kickoff_async(
    inputs=inputs
))

## Stakeholder Specific Modeling

In [11]:
from echo import setup_db_tables
import nest_asyncio
from dotenv import load_dotenv
import asyncio
from echo.constants import (
    DISCOVERY,
    DEMO,
    PRICING,
    NEGOTIATION
)
from echo.runner import create_or_get_seller

nest_asyncio.apply()

load_dotenv()


setup_db_tables()

NUM_BUYERS = 10

inputs = {
    "seller": "foundit.in",
    "num_buyers": NUM_BUYERS,
}

seller_data = asyncio.run(create_or_get_seller(inputs))

Tables Created Successfully
Creating seller for foundit.in
Seller Research Data Found
Adding Seller: foundit.in Research Data
Node already exists in index: {'seller': 'foundit.in', 'industry': 'Recruitment and Job Search', 'data': '{"seller_research": {"name": "Foundit.in", "website": "https://www.foundit.in/", "description": "India\'s leading job portal helping job seekers to find jobs and employers to find talent", "industry": "Recruitment and Job Search", "value_propositions": [{"product": "Talent Search", "solution": "A tool to navigate an ocean of profiles", "use_cases": ["Helps you locate talent that\'s a perfect fit", "Streamlines your recruitment process"]}, {"product": "Resume Search", "solution": "A platform to search and filter resumes", "use_cases": ["Helps recruiters to find the right candidate", "Saves time in the recruitment process"]}, {"product": "Magic Search", "solution": "An AI-driven recruitment solution", "use_cases": ["Transforms your hiring process", "Helps you 

In [12]:
# from echo.indexing import IndexType, get_metadatas
# from echo.indexing import get_vector_index


# vector_index = get_vector_index("foundit.in", IndexType.SELLER_RESEARCH.value)
# get_metadatas(vector_index)

In [13]:
from echo.runner import make_call


buyer_inputs = {
    **inputs,
    **seller_data,
    'n_competitors': 10,
    'call_id': 1,
    'stakeholders': [
        "Product Manager",
        "CFO",
        "VP of Product",
        "VP of Sales"
    ]
}

clients = seller_data['seller_clients']['clients']
seller_data['seller_clients']['clients']

['Capgemini',
 'TATA Consultancy Services',
 'LTI',
 'American book writer',
 'State Street Corporation',
 'Epiq Hyderabad',
 'ZUNO_PROD-AZ']

In [14]:
discovery_calls_data = asyncio.run(make_call(DISCOVERY, seller_data['seller_clients']['clients'][:5], buyer_inputs))

Making discovery call for ['Capgemini', 'TATA Consultancy Services', 'LTI', 'American book writer', 'State Street Corporation']
Getting analysis Data
Number of Clients:  5


Getting Data:   0%|          | 0/5 [00:00<?, ?it/s]

Getting Data for Capgemini
Seller Research Data Found
Adding Seller: foundit.in Research Data
Node already exists in index: {'seller': 'foundit.in', 'industry': 'Recruitment and Job Search', 'data': '{"seller_research": {"name": "Foundit.in", "website": "https://www.foundit.in/", "description": "India\'s leading job portal helping job seekers to find jobs and employers to find talent", "industry": "Recruitment and Job Search", "value_propositions": [{"product": "Talent Search", "solution": "A tool to navigate an ocean of profiles", "use_cases": ["Helps you locate talent that\'s a perfect fit", "Streamlines your recruitment process"]}, {"product": "Resume Search", "solution": "A platform to search and filter resumes", "use_cases": ["Helps recruiters to find the right candidate", "Saves time in the recruitment process"]}, {"product": "Magic Search", "solution": "An AI-driven recruitment solution", "use_cases": ["Transforms your hiring process", "Helps you find the best fit for your organ

Getting Data:  60%|██████    | 3/5 [00:00<00:00,  9.36it/s]

Node already exists in index: {'buyer': 'Capgemini', 'seller': 'foundit.in', 'industry': 'IT Services and IT Consulting', 'company_size': 'Enterprise (10,001+ employees)', 'data': '{"buyer_research": {"name": "Capgemini", "website": "https://www.capgemini.com/", "description": "A global leader in consulting, technology services and digital transformation", "industry": "IT Services and IT Consulting", "company_size": "Enterprise (10,001+ employees)", "goals": ["Accelerate business transformation", "Unlock innovation and drive business growth", "Deliver exceptional experiences"], "use_cases": ["Digital transformation", "Innovation and growth", "Operational efficiency"], "challenges": ["Staying ahead of the competition", "Adapting to changing market conditions", "Meeting customer expectations"], "stakeholders": ["Business leaders", "IT professionals", "Customers"]}, "competitive_info": {"competitors": [{"name": "Naukri", "pros": ["Large database of job postings", "Trusted name in the Indi

Getting Data: 100%|██████████| 5/5 [00:00<00:00, 10.55it/s]

Node already exists in index: {'seller': 'foundit.in', 'buyer': 'American book writer', 'call_type': 'discovery', 'company_size': 'SMB', 'industry': 'Book Writing and Publishing', 'description': "American Book Writer is a California-based Authors' favorite website to get your book written and published smoothly.", 'stakeholder': 'CFO', 'transcript': '{"messages": [{"user": "Sales Person (foundit.in)", "content": "Hi everyone, thank you for taking the time to speak with me today. I\'m excited to learn more about American Book Writer\'s goals and challenges. Can each of you please introduce yourselves and tell me a little bit about your roles?"}, {"user": "Product Manager (American Book Writer)", "content": "Hi, I\'m John, the Product Manager. I oversee the development of our book writing services and ensure they meet the needs of our authors."}, {"user": "CFO (American Book Writer)", "content": "Hi, I\'m David, the CFO. I\'m responsible for managing the financial aspects of our business

In [15]:
buyer_inputs['call_id'] = 2
demo_calls_data = asyncio.run(make_call(DEMO, seller_data['seller_clients']['clients'][:5], buyer_inputs))

Making demo call for ['Capgemini', 'TATA Consultancy Services', 'LTI', 'American book writer', 'State Street Corporation']
Getting analysis Data
Number of Clients:  5


Getting Data:   0%|          | 0/5 [00:00<?, ?it/s]

Getting Data for Capgemini
Adding Buyer: Capgemini Data
Node already exists in index: {'buyer': 'Capgemini', 'seller': 'foundit.in', 'industry': 'IT Services and IT Consulting', 'company_size': 'Enterprise (10,001+ employees)', 'data': '{"demo_features": {"features": [{"name": "Talent Search", "description": "A tool to navigate an ocean of profiles, helps you locate talent that\'s a perfect fit and streamlines your recruitment process"}, {"name": "Resume Search", "description": "A platform to search and filter resumes, helps recruiters to find the right candidate and saves time in the recruitment process"}, {"name": "Magic Search", "description": "An AI-driven recruitment solution that transforms your hiring process and helps you find the best fit for your organization"}, {"name": "Job Posting Template", "description": "A tool to help employers create high-quality job postings"}, {"name": "Guidance on Writing Effective Job Descriptions", "description": "A feature to help employers writ

Getting Data:  40%|████      | 2/5 [00:00<00:00, 14.27it/s]

Node already exists in index: {'seller': 'foundit.in', 'buyer': 'TATA Consultancy Services', 'call_type': 'demo', 'company_size': 'Enterprise', 'industry': 'IT Services and IT Consulting', 'description': "Tata Consultancy Services is an IT services, consulting and business solutions organization that partners with many of the world's largest businesses.", 'stakeholder': 'VP of Product', 'transcript': '{"messages": [{"user": "foundit.in Sales Person", "content": "Hello everyone, thank you for taking the time to speak with me today about foundit.in\'s solutions. I\'d like to start by briefly introducing our platform and how it can address the pain points we discussed during our discovery call."}, {"user": "VP of Sales", "content": "Yes, please do. We\'re interested in learning more about how foundit.in can help us find the right candidates and reduce our time-to-hire."}, {"user": "foundit.in Sales Person", "content": "Great. foundit.in\'s platform is designed to streamline the recruitmen

Getting Data: 100%|██████████| 5/5 [00:00<00:00, 17.34it/s]

Node already exists in index: {'seller': 'foundit.in', 'buyer': 'American book writer', 'call_type': 'demo', 'transcript': '{"messages": [{"user": "Foundit.in Salesperson", "content": "Hello everyone, thank you for taking the time to join this demo call today. I\'m excited to show you how Foundit.in can help American Book Writer address its recruitment challenges. Before we dive in, can each of you tell me a little bit about your roles and how you\'re involved in the recruitment process?"}, {"user": "VP of Sales", "content": "I\'m Michael, VP of Sales. I oversee the sales team and our revenue growth. I\'m interested in seeing how Foundit.in can help us improve our recruitment process and reduce costs."}, {"user": "CFO", "content": "I\'m David, CFO. I\'m responsible for our budget and financial planning. I\'m concerned about the high costs of our current recruitment process and would like to see how Foundit.in can help us reduce those costs."}, {"user": "VP of Product", "content": "I\'m

In [16]:
from echo import sqldb
from echo.indexing import IndexType

len(sqldb.query_records(
    IndexType.BUYER_RESEARCH.value,
))

5

In [17]:
for i in sqldb.query_records(
    IndexType.BUYER_RESEARCH.value,
):
    print(i['buyer'])

State Street Corporation
American book writer
LTI
TATA Consultancy Services
Capgemini


In [18]:
from echo.indexing import get_metadatas, get_vector_index


vector_index = get_vector_index("foundit.in", IndexType.BUYER_RESEARCH.value)
metadatas = get_metadatas(vector_index)
for i in metadatas:
    print(i['buyer'])

Capgemini
Capgemini
TATA Consultancy Services
LTI
American book writer
State Street Corporation
TATA Consultancy Services
LTI
American book writer
State Street Corporation


In [19]:
buyer_inputs['call_id'] = 3
pricing_calls_data = asyncio.run(make_call(PRICING, seller_data['seller_clients']['clients'][:5], buyer_inputs))

Making pricing call for ['Capgemini', 'TATA Consultancy Services', 'LTI', 'American book writer', 'State Street Corporation']
Getting analysis Data
Number of Clients:  5


Getting Data:   0%|          | 0/5 [00:00<?, ?it/s]

Getting Data for Capgemini
Embedding Simulation Data for Client: Capgemini
Node already exists in index: {'seller': 'foundit.in', 'buyer': 'Capgemini', 'call_type': 'pricing', 'transcript': '{"messages": [{"user": "Sales Person", "content": "Good morning, everyone. Thank you for taking the time to discuss our pricing models today. I have all the stakeholders on the call, including the VP of Product, CFO, VP of Sales, and Product Manager. Is that correct?"}, {"user": "VP of Product", "content": "Yes, that\'s correct. We\'re all here."}, {"user": "Sales Person", "content": "Great. Before we dive into our pricing models, I want to confirm that you\'ve all had a chance to review the demo we presented earlier this week. Do you have any questions about the features and benefits of our Magic Search and Resume Search platform?"}, {"user": "Product Manager", "content": "Actually, yes. I did have a question about the quality of job postings on your platform. How do you ensure that the job postin

Getting Data:  60%|██████    | 3/5 [00:00<00:00, 17.82it/s]

Node already exists in index: {'seller': 'foundit.in', 'buyer': 'LTI', 'call_type': 'pricing', 'company_size': 'Enterprise', 'industry': 'Technology, Consulting', 'description': 'Larsen & Toubro Infotech (LTI) is a global technology consulting and digital solutions company that enables enterprises across industries to reimagine business models, accelerate innovation, and maximize growth by harnessing digital technologies.', 'stakeholder': 'VP of Sales', 'transcript': '{"messages": [{"user": "foundit.in\'s Sales Person", "content": "Hello everyone, thank you for taking the time to discuss the pricing models for our product. I believe we had a great demo discussion last week, and I\'m excited to provide you with our pricing options that can help LTI achieve its recruitment goals."}, {"user": "Rohan, Product Manager", "content": "Hi, yes, we were impressed with the demo. Can you please walk us through your pricing models and how they can help us streamline our recruitment process?"}, {"us

Getting Data: 100%|██████████| 5/5 [00:00<00:00, 19.74it/s]

Node already exists in index: {'seller': 'foundit.in', 'buyer': 'State Street Corporation', 'call_type': 'pricing', 'company_size': 'Enterprise', 'industry': 'Financial services', 'description': 'State Street provides investment servicing, investment management, investment research and trading services to institutional investors worldwide.', 'stakeholder': 'VP of Sales', 'transcript': '{"messages": [{"user": "Sales Person (foundit.in)", "content": "Hello everyone, thank you for taking the time to discuss pricing for our solution. I\'m excited to walk you through our different pricing models and answer any questions you may have."}, {"user": "CFO (State Street Corporation)", "content": "Thanks for having us. We\'re interested in learning more about your pricing models and how they can help us achieve our recruitment goals."}, {"user": "Sales Person (foundit.in)", "content": "Absolutely. We have four main pricing models: Free, Premium, Career Booster, and Featured Profile. Our Free model

In [20]:
buyer_inputs['call_id'] = 4
negotiations_calls_data = asyncio.run(make_call(NEGOTIATION, seller_data['seller_clients']['clients'][:5], buyer_inputs))

Making negotiation call for ['Capgemini', 'TATA Consultancy Services', 'LTI', 'American book writer', 'State Street Corporation']
Getting analysis Data
Number of Clients:  5


Getting Data:   0%|          | 0/5 [00:00<?, ?it/s]

Getting Data for Capgemini
Embedding Simulation Data for Client: Capgemini
Node already exists in index: {'seller': 'foundit.in', 'buyer': 'Capgemini', 'call_type': 'negotiation', 'transcript': '{"messages": [{"user": "foundit.in", "content": "Hi everyone, thank you for taking the time to discuss the proposal we sent over. I\'d like to summarize the key points and address any concerns you may have. As we discussed earlier, our Premium plan would provide Capgemini with enhanced services to enhance the job search journey, AI-driven resume matching, priority customer support, and access to our recruitment analytics platform."}, {"user": "VP of Product", "content": "Yes, I think we\'re all on the same page in terms of the services. However, we did have some concerns about the quality of job postings on the platform. Can you elaborate on how you\'ll ensure that the job postings meet our standards?"}, {"user": "foundit.in", "content": "Absolutely. We have a team of moderators who review each

Getting Data:  60%|██████    | 3/5 [00:00<00:00, 24.40it/s]

Node already exists in index: {'seller': 'foundit.in', 'buyer': 'LTI', 'call_type': 'negotiation', 'company_size': 'Enterprise', 'industry': 'Technology, Consulting', 'description': 'Larsen & Toubro Infotech (LTI) is a global technology consulting and digital solutions company that enables enterprises across industries to reimagine business models, accelerate innovation, and maximize growth by harnessing digital technologies.', 'stakeholder': 'CFO', 'transcript': '{"messages": [{"user": "foundit.in", "content": "Hi everyone, thank you for taking the time to discuss the implementation of our recruitment solution. I understand we previously discussed our pricing models, and I\'d like to address any concerns you may have and finalize the terms of our agreement."}, {"user": "VP of Sales", "content": "Yes, we\'re interested in moving forward, but we need to discuss the duration of the plans. We\'re concerned that the Premium model is only available for a year, and we\'d like to explore long

Getting Data: 100%|██████████| 5/5 [00:00<00:00, 25.29it/s]

Node already exists in index: {'seller': 'foundit.in', 'buyer': 'State Street Corporation', 'call_type': 'negotiation', 'company_size': 'Enterprise', 'industry': 'Financial services', 'description': 'State Street provides investment servicing, investment management, investment research and trading services to institutional investors worldwide.', 'stakeholder': 'VP of Sales', 'transcript': '{"messages": [{"user": "foundit.in", "content": "Hi everyone, thank you for taking the time to discuss the next steps for our partnership. I\'m excited to move forward with implementing our solution to help State Street Corporation reduce hiring costs and improve time-to-hire."}, {"user": "VP of Sales", "content": "Yes, we\'re interested in moving forward, but we did have some concerns about the integration process with our existing HR systems. Can you walk us through how that would work?"}, {"user": "foundit.in", "content": "Absolutely. We\'ve had success with similar integrations in the past. Our i

In [21]:
from echo.queries import get_queries

seller = "foundit.in"

In [22]:
from echo.query_executor import SubQuery, Query
from echo.step_templates.generic import CallType
from echo.indexing import IndexType


query = Query(
    seller=seller,
    call_type=CallType.DISCOVERY.value,
    query=(
        "Given all the information needed to uncover and the current information uncovered till now - "
        "What all pieces of information are missing that should be uner and the current information uncovered till now"
        "What all pieces of information are missing that should be uncovered in upcoming calls?"
    ),
    sub_queries=[
        SubQuery(
            query="What all pieces of information about the buyer are uncovered in discovery calls of successful deals?",
            index_type=IndexType.ANALYSIS.value,
            inputs = {"stakeholder": "CFO"}
        ),
        SubQuery(
            query="What all piece of information have we learnt about the buyer from discovery till now?",
            index_type=IndexType.CURRENT_CALL.value,
        ),
    ],
)

In [23]:
### Check what all values you need to provide for any index type

from echo.indexing import get_query_index_keys

index_type = IndexType.ANALYSIS.value

def show_required_inputs(index_type):
    index_keys = get_query_index_keys(index_type)
    for index_key in index_keys:
        print(f"{index_key['key']} {'required' if index_key['mandatory'] else 'optional'}")

show_required_inputs(index_type)

seller required
buyer required
call_type required
stakeholder optional


In [24]:
from echo.query_executor import aget_query_response


response = asyncio.run(
    aget_query_response(
        query,
        inputs={
            "buyer": "Capgemini",
            "seller": seller,
            "call_type": "discovery",
        }
    )
)

Running sub query What all pieces of information about the buyer are uncovered in discovery calls of successful deals?
Sub query context {'query': 'What all pieces of information about the buyer are uncovered in discovery calls of successful deals?', 'context': 'Relevant Context:\nThe pain points, objections, success indicators, budget constraints, competition, and decision committee members are the pieces of information about the buyer that are uncovered in discovery calls of successful deals.'}
Running sub query What all piece of information have we learnt about the buyer from discovery till now?
Sub query context {'query': 'What all piece of information have we learnt about the buyer from discovery till now?', 'context': "Relevant Context:\nThe buyer, Capgemini, has expressed pain points related to difficulty in finding the right candidates and having a time-consuming and expensive recruitment process. They have objections regarding the quality of job postings on the platform. Capge

In [25]:
print(response[0])

### Missing Pieces of Information to Uncover in Upcoming Calls

#### Pain Points and Challenges

* More specific details about the current recruitment process and how it is impacting the business
* Quantifiable metrics on the current time-to-hire and cost-per-hire
* Other pain points aside from finding the right candidates and having a time-consuming and expensive recruitment process

#### Goals and Success Indicators

* Clear Key Performance Indicators (KPIs) that the buyer wants to achieve with a new recruitment solution
* Specific metrics for ROI, time-to-hire, and cost-per-hire that the buyer wants to achieve
* Other goals aside from streamlining the recruitment process, reducing costs, and staying ahead of the competition

#### Budget Constraints

* Specific budget constraints for a new recruitment solution
* Any limits on the budget for recruitment technology and services
* Flexibility in the budget for a solution that can demonstrate clear ROI

#### Decision Committee and Stakeh